# Torre de Hanoi - Algoritmo A*
# Introducción a la Inteligencia Artificial

## Problema de la Torre de Hanoi con 5 discos

### Descripción del problema
La Torre de Hanoi es un problema clásico que consiste en mover una torre de discos de diferentes tamaños de una varilla a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez
2. Un disco solo puede colocarse sobre otro disco más grande
3. Objetivo: mover todos los discos de la varilla izquierda (A) a la derecha (C)

### Estado inicial: Todos los discos en la varilla A
### Estado objetivo: Todos los discos en la varilla C

## Importamos las librerias necesarias

In [575]:
import time
import json
from typing import List, Tuple
import tracemalloc
import statistics
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

## Definimos el problema

In [576]:
class TowerHanoiAStar:
    """
    Implementación del algoritmo A* para resolver la Torre de Hanoi.
    """

    def __init__(self, initial_state: StatesHanoi, goal_state: StatesHanoi, problem: ProblemHanoi, disks_num: int = 5):
        self.disks_num = disks_num
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.problem = problem
        self.explored_nodes = 0
        self.expanded_nodes = 0

    def multifactorial_heuristic(self, node: NodeHanoi) -> int:
        """
        Heurística multifactorial: Combinación de múltiples factores

        Fundamentación:
        1. Discos mal ubicados: Penaliza discos que no están en la varilla objetivo
        2. Discos bloqueados: Penaliza discos que tienen otros encima en varillas incorrectas
        3. Orden incorrecto: Penaliza cuando los discos no están en orden correcto
        4. Distancia de movimiento: Considera qué tan lejos están los discos de su destino

        Esta heurística es admisible porque nunca sobreestima el costo real,
        ya que cada factor representa movimientos mínimos necesarios.
        """
        # Factor 1: Discos que no están en la varilla objetivo (C)
        misplaced_disks = 0
        for i in range(2):  # Varillas A y B
            misplaced_disks += len(node.state.rods[i])

        # Factor 2: Discos bloqueados en varillas incorrectas
        locked_disks = 0
        for i in range(2):  # Solo varillas A y B
            rod = node.state.rods[i]
            for j, _ in enumerate(rod):
                # Si hay discos encima, está bloqueado
                if j < len(rod) - 1:
                    locked_disks += 1

        # Factor 3: Verificar orden en varilla objetivo
        incorrect_order = 0
        rod_c = node.state.rods[2]
        for i in range(len(rod_c)):
            if rod_c[i] != self.disks_num - i:
                incorrect_order += 1

        # Factor 4: Peso por tamaño de disco (discos más grandes son más costosos de mover)
        disks_weight = 0
        for i in range(2):
            for disk in node.state.rods[i]:
                disks_weight += disk * 0.1  # Pequeño peso adicional

        # Combinación de factores
        h = misplaced_disks * 2 + locked_disks + incorrect_order + disks_weight

        return int(h)

    def simple_heuristic(self, node: NodeHanoi) -> int:
        """
        Heurística simple propuesta: -1 por cada disco en posición correcta
        (Convertida a positiva para A*)
        """
        correct_disks = len(node.state.rods[2])  # Discos en varilla C
        return self.disks_num - correct_disks

    def a_star(self, use_multifactorial_heuristic: bool = True, debug: bool = False) -> Tuple[NodeHanoi, dict]:
        """
        Implementación del algoritmo A*.

        Returns:
            - Nodo final alcanzado
            - Diccionario con estadísticas del algoritmo
        """
        if debug:
            print(f"Iniciando A* con heurística {'multifactorial' if use_multifactorial_heuristic else 'simple'}...")
            print(f"Estado inicial: {self.initial_state}")
            print(f"Estado objetivo: {self.goal_state}")
            print("-" * 60)

        start_time = time.time()

        # Inicialización
        heuristic = self.multifactorial_heuristic if use_multifactorial_heuristic else self.simple_heuristic
        root = NodeHanoi(state=self.initial_state)
        priority_queue = AimaPriorityQueue(order='min', f=lambda node: (node.path_cost + heuristic(node)))

        priority_queue.append(root)
        open_states = {root.state} # Usamos un set para estados en la lista abierta
        closed_set = set()  # Usamos set en lugar de list para closed
        state_costs = {root.state: root.path_cost} # Diccionario para mantener track del mejor costo para cada estado
        self.explored_nodes = 0
        self.expanded_nodes = 0

        while len(priority_queue) != 0:

            # Seleccionar nodo con menor f
            priority_value, current_node = priority_queue.pop()
            current_state = current_node.state

            # Si este nodo tiene un costo peor que el que ya conocemos, lo saltamos
            if current_state in state_costs and current_node.path_cost > state_costs[current_state]:
                continue

            # Removemos el estado de open_states
            open_states.remove(current_state)

            self.explored_nodes += 1

            # Verificar si alcanzamos el objetivo
            if current_state == self.goal_state:
                total_time = time.time() - start_time

                # Reconstruir camino
                stats = {
                    'movimientos': len(current_node.path()) - 1,
                    'nodos_explorados': self.explored_nodes,
                    'nodos_expandidos': self.expanded_nodes,
                    'tiempo_ejecucion': total_time,
                    'costo_solucion': current_node.path_cost,
                    'heuristica_usada': 'miltifactorial' if use_multifactorial_heuristic else 'simple'
                }

                return current_node, stats

            # Agregamos a closed_set el estado, no el nodo
            closed_set.add(current_state)

            # Expandir nodos sucesores
            self.expanded_nodes += 1

            # Expandir nodos sucesores
            for next_node in current_node.expand(problem=self.problem):
                next_state = next_node.state

                # Si el estado está cerrado y no encontramos un mejor camino, lo saltamos
                if next_state in closed_set and next_node.path_cost >= state_costs.get(next_state, float('inf')):
                    continue

                # Si encontramos un mejor camino a un estado
                if next_state not in state_costs or next_node.path_cost < state_costs[next_state]:
                    # Actualizamos el mejor costo conocido
                    state_costs[next_state] = next_node.path_cost
                    # Agregamos el nodo a la cola con la nueva prioridad
                    priority_queue.append(next_node)
                    open_states.add(next_state)
                    # Si estaba en closed_set, lo removemos para reconsiderarlo
                    if next_state in closed_set:
                        closed_set.remove(next_state)

        return None, {'error': 'No se encontró solución'}

# Definimos métodos auxiliares

In [577]:
def compare_heuristics(disks_num: int = 5, debug: bool = False) -> Tuple[NodeHanoi, dict]:
    """Inicialización del problema"""
    disks_list = list(range(disks_num, 0, -1))
    initial_state = StatesHanoi(disks_list, [], [], max_disks=disks_num)
    goal_state = StatesHanoi([], [], disks_list, max_disks=disks_num)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Probar heurística simple
    tower = TowerHanoiAStar(initial_state, goal_state, problem, disks_num=disks_num)
    solution_simple, stats_simple = tower.a_star(use_multifactorial_heuristic=False)

    if solution_simple and debug:
        print("\n1. HEURÍSTICA SIMPLE (discos fuera de lugar)")
        print(f"✓ Solución encontrada en {stats_simple['movimientos']} movimientos")
        print(f"  - Nodos explorados: {stats_simple['nodos_explorados']}")
        print(f"  - Nodos expandidos: {stats_simple['nodos_expandidos']}")
        print(f"  - Tiempo: {stats_simple['tiempo_ejecucion']:.4f} segundos")

    # Probar heurística multifactorial
    tower = TowerHanoiAStar(initial_state, goal_state, problem, disks_num=disks_num)  # Nueva instancia
    solution_multifactor, stats_multifactor = tower.a_star(use_multifactorial_heuristic=True)

    if solution_multifactor and debug:
        print("\n2. HEURÍSTICA MULTIFACTORIAL")
        print(f"✓ Solución encontrada en {stats_multifactor['movimientos']} movimientos")
        print(f"  - Nodos explorados: {stats_multifactor['nodos_explorados']}")
        print(f"  - Nodos expandidos: {stats_multifactor['nodos_expandidos']}")
        print(f"  - Tiempo: {stats_multifactor['tiempo_ejecucion']:.4f} segundos")

    # Análisis comparativo
    print("\n" + "="*50)
    print("ANÁLISIS COMPARATIVO")
    print("="*50)

    if solution_simple and solution_multifactor:
        if stats_simple['movimientos'] == stats_multifactor['movimientos'] and stats_multifactor['movimientos'] == (2 ** disks_num - 1):
                print(f"Ambas heuristicas encontraron solución óptima: {stats_simple['movimientos']} movimientos")
        print(f"Eficiencia en nodos explorados:")
        print(f"  - Simple: {stats_simple['nodos_explorados']} nodos")
        print(f"  - Creativa: {stats_multifactor['nodos_explorados']} nodos")
        print(f"  - Mejora: {((stats_simple['nodos_explorados'] - stats_multifactor['nodos_explorados']) / stats_simple['nodos_explorados'] * 100):.1f}%")

    return solution_multifactor, stats_multifactor

In [578]:
def validate_files() -> bool:
    """
    Valida que los archivos generados tengan el formato correcto.
    """

    # 1. Validar initial_state.json
    print("1. Validando initial_state.json...")
    try:
        with open('initial_state.json', 'r', encoding='utf-8') as archivo:
            initial_state = json.load(archivo)

        # Verificar estructura
        required_keys = ['peg_1', 'peg_2', 'peg_3']
        if not all(key in initial_state for key in required_keys):
            print("   ✗ Estructura incorrecta: faltan claves requeridas")
            return False

        # Verificar que todos los valores sean listas
        for key in required_keys:
            if not isinstance(initial_state[key], list):
                print(f"   ✗ {key} debe ser una lista")
                return False

        # Validar discos
        todos_discos = []
        for peg in ['peg_1', 'peg_2', 'peg_3']:
            todos_discos.extend(initial_state[peg])

        # Verificar discos únicos
        if len(todos_discos) != len(set(todos_discos)):
            print("   ✗ Hay discos duplicados")
            return False

        # Verificar secuencia completa
        if todos_discos and (set(todos_discos) != set(range(1, max(todos_discos) + 1))):
            print("   ✗ Secuencia de discos incompleta")
            return False

        print("   ✓ initial_state.json válido")

    except FileNotFoundError:
        print("   ✗ Archivo initial_state.json no encontrado")
        return False
    except json.JSONDecodeError:
        print("   ✗ Error al decodificar initial_state.json")
        return False

    # 2. Validar sequence.json
    print("2. Validando sequence.json...")
    try:
        with open('sequence.json', 'r', encoding='utf-8') as archivo:
            sequence = json.load(archivo)

        # Verificar que sea una lista
        if not isinstance(sequence, list):
            print("   ✗ sequence.json debe ser una lista")
            return False

        # Validar cada movimiento
        for i, move in enumerate(sequence):
            if not isinstance(move, dict):
                print(f"   ✗ Movimiento {i} debe ser un objeto")
                return False

            required_move_keys = ['type', 'disk', 'peg_start', 'peg_end']
            if not all(key in move for key in required_move_keys):
                print(f"   ✗ Movimiento {i} incompleto")
                return False

            if move['type'] != 'movement':
                print(f"   ✗ Movimiento {i} debe tener type='movement'")
                return False

            # Verificar que los números de varilla sean válidos
            if not (1 <= move['peg_start'] <= 3 and 1 <= move['peg_end'] <= 3):
                print(f"   ✗ Movimiento {i} tiene números de varilla inválidos")
                return False

        print(f"   ✓ sequence.json válido ({len(sequence)} movimientos)")

    except FileNotFoundError:
        print("   ✗ Archivo sequence.json no encontrado")
        return False
    except json.JSONDecodeError:
        print("   ✗ Error al decodificar sequence.json")
        return False

    print("\n🎉 Ambos archivos son válidos!")
    return True

In [579]:
def get_execution_time_and_memory_usage(iterations: int = 10, use_multifactorial_heuristic: bool = True) -> Tuple[List[float], List[float], int]:
    """
    Cálculo del tiempo de ejecución y el uso de memoria requeridos para la búsqueda de una solución.
    Calcula el promedio y el desvío estándar de ambas métricas.

    Returns:
        - Lista con el promedio y el desvío estándar de los tiempos de ejecución
        - Lista con el promedio y el desvío estándar de los usos de memoria
        - Promedio de movimientos en la solución
    """

    """Inicialización del problema"""
    n = 5 # número de discos de la torre
    disks_list = list(range(n, 0, -1))
    initial_state = StatesHanoi(disks_list, [], [], max_disks=n)
    goal_state = StatesHanoi([], [], disks_list, max_disks=n)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)
    tower = TowerHanoiAStar(initial_state, goal_state, problem, disks_num=n)

    time_data = []
    memory_data = []
    movements = []
    for _ in range(iterations):
        tracemalloc.start()
        solution_multifactor, stats_multifactor = tower.a_star(use_multifactorial_heuristic=use_multifactorial_heuristic)
        _, memory_peak = tracemalloc.get_traced_memory()
        memory_peak /= 1024 * 1024
        tracemalloc.stop()
        time_data.append(stats_multifactor['tiempo_ejecucion'])
        memory_data.append(memory_peak)
        movements.append(stats_multifactor['movimientos'])

    time_list = [statistics.mean(time_data), statistics.stdev(time_data)]
    memory_list = [statistics.mean(memory_data), statistics.stdev(memory_data)]
    return time_list, memory_list, statistics.mean(movements)

# EJECUCIÓN PRINCIPAL

In [580]:
solution, stats = compare_heuristics(disks_num=5, debug=False)

solution.generate_solution_for_simulator()
print(f"Algoritmo: A* con heurística {stats.get('heuristica_usada', 'miltifactorial')}")
print(f"   Tiempo de ejecución: {stats.get('tiempo_ejecucion', 0):.4f} segundos")
print(f"   Nodos explorados: {stats.get('nodos_explorados', 0)}")
#validate_files()


ANÁLISIS COMPARATIVO
Ambas heuristicas encontraron solución óptima: 31 movimientos
Eficiencia en nodos explorados:
  - Simple: 178 nodos
  - Creativa: 154 nodos
  - Mejora: 13.5%
Algoritmo: A* con heurística miltifactorial
   Tiempo de ejecución: 0.0092 segundos
   Nodos explorados: 154


## Ejecutamos 10 veces para evaluar cuánto tiempo y memoria utiliza el algoritmo

In [581]:
time_stats, memory_stats, movements = get_execution_time_and_memory_usage(iterations=10, use_multifactorial_heuristic=True)
print("TIEMPO DE EJECUCIÓN:")
print(f"   Promedio de tiempo de ejecución: {time_stats[0]:.4f} segundos")
print(f"   Desviación estándar de tiempo de ejecución: {time_stats[1]:.4f} segundos")
print("\nUSO DE MEMORIA:")
print(f"   Promedio de uso de memoria: {memory_stats[0]:.4f} MB")
print(f"   Desviación estándar de uso de memoria: {memory_stats[1]:.4f} MB")
print("\nMOVIMIENTOS:")
print(f"   Promedio de movimientos: {movements}")

TIEMPO DE EJECUCIÓN:
   Promedio de tiempo de ejecución: 0.0342 segundos
   Desviación estándar de tiempo de ejecución: 0.0007 segundos

USO DE MEMORIA:
   Promedio de uso de memoria: 0.1727 MB
   Desviación estándar de uso de memoria: 0.0047 MB

MOVIMIENTOS:
   Promedio de movimientos: 31
